In [ ]:
import math
from pyspark.sql import SparkSession

Spark = SparkSession.builder.appName("Day").getOrCreate()
SparkContext = Spark.sparkContext

In [ ]:
# @title Ejercicio 1
#Creando RDD llamado lenguajes
lenguajes = SparkContext.parallelize(["Python", "R", "C", "Scala", "Rugby", "SQL"])

#Filtrar lenguajes con la letra R
lenguajes_R = lenguajes.filter(lambda x: x.startswith("R"))
print("Los lenguajes que inician con la letra 'R' son : ", lenguajes_R.collect())

In [ ]:
# @title Ejercicio 2 - A
# Creando RDD llamado pares con rango de 20 a 31 porque si no, no toma el 30
# El 2 es porque irá de 2 en 2, osea pares
pares = SparkContext.parallelize(range(20, 31, 2))

# Obtener una lista compuesta por los números pares en el intervalo [20;30] y sus respectivas raíces cuadradas
pares_raices = pares.map(lambda x: (x, math.sqrt(x))).collect()

#
print("Números pares y su raiz cuadrada:")
for numeritos in pares_raices:
    print(numeritos)

#En este caso use flatmap para mostrarlo como sin los () y la formula de raiz cuadrada
pares_raices2 = pares.flatMap(lambda x: (x, x ** 0.5))
print("\nRaiz Cuadrada de otra forma:")
print(pares_raices2.collect())

In [ ]:
# @title Ejercicio 2 - B
# Reducir el número de particiones a 20, en este caso no afecta porque la cantidad de datos es poca, pero si está definido
pares_raices2 = pares_raices2.coalesce(20)

# Recoger los resultados
resultados_pares_raices2 = pares_raices2.collect()

# Imprimir los resultados
print("Números pares y su raiz cuadrada:")
for numeritos in resultados_pares_raices2:
    print(numeritos)


In [ ]:
# @title Ejercicio 2 - C
ejercicio2_c = " Se podría usar repartition() que tambien  también puede reducir el número de particiones \n Aunque no es tan eficiente como coalesce() cuando el objetivo es disminuir el número de particiones, \n especialmente si el número actual de particiones es mayor que el número deseado de particiones "
print(ejercicio2_c)

In [ ]:
# @title Ejercicio 3
from pyspark.sql import SparkSession

# Crear o obtener la sesión de Spark
spark = SparkSession.builder.appName("TotalPorCuenta").getOrCreate()
sc = spark.sparkContext

# Ruta al archivo de datos
filepath = 'transacciones'

# Cargar el archivo de texto
lines = sc.textFile(filepath)

# Función para convertir cada línea en una tupla (número de cuenta, monto)
def parse_line(line):
    parts = line.strip().strip('()').split(',')
    account = int(parts[0])
    amount = float(parts[1])
    return (account, amount)

# Crear el RDD de clave-valor
transactions = lines.map(parse_line)

# Sumar los montos por cuenta
total_por_cuenta = transactions.reduceByKey(lambda a, b: a + b)

# Mostrar los resultados
for cuenta, total in total_por_cuenta.collect():
    print(f"Cuenta: {cuenta}, Total: {total}")